<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo3/2_aplicaciones_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1e7ctPi8O3bTQoLZaO9ZZjwGr2r8Z93RS" width="100%">

# **Aplicaciones de las Bolsas de Palabras**
---

En este notebook veremos algunas aplicaciones que pueden realizarse con representaciones de bolsas de palabras, en específico hablaremos de resúmenes automáticos de texto, léxicos y nubes de palabras. Comenzaremos importando las librerías de ciencia de datos necesarias para manejo de datos, visualización, y manipulación de strings.

In [ ]:
!pip install unidecode

In [ ]:
import re
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from IPython.display import display
plt.style.use("ggplot")
spacy.cli.download("es_core_news_sm")

Vamos a preprocesar los documentos con una función de preprocesamiento similar a la que usamos en el notebook de bolsa de palabras, no obstante, en este caso también eliminaremos las _stopwords_:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")
nlp = spacy.load(
        "es_core_news_sm",
        exclude=[
            "attribute_ruler", "lemmatizer", "ner"
            ]
        )
def preprocess(text, min_len=1, max_len=23):
    # Normalizamos el texto
    norm_text = unidecode(text).lower()

    # Extraemos tokens
    tokens = nlp(norm_text)

    # Filtramos palabras por longitud
    filtered_tokens = filter(
            lambda token: (
                len(token) >= min_len and
                len(token) <= max_len and
                not token.is_stop  # Filtramos stopwords
                ),
            tokens
        )
    filtered_text = " ".join(token.text for token in filtered_tokens)
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", filtered_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

## **1. Resumen Extractivo de Texto**
---

Una de las aplicaciones más típicas de las representaciones de bolsas de palabras es el resumen extractivo de texto. Se trata de una aplicación muy típica relacionada con el procesamiento de lenguaje natural, donde se busca resumir la información que hay en un documento, obteniendo una versión corta que conserve la mayor cantidad de información posible del texto original.

<img src="https://drive.google.com/uc?export=view&id=1s5Mq_j5QIPatLwoliojU2YvKBgWl003n" width="100%">

Normalmente el resumen automático de texto se aborda desde dos enfoques:

- **Extractivo**: consiste en extraer los _chunks_ o fragmentos del texto con mayor importancia.
- **Abstractivo**: consiste en extraer una síntesis del texto que no necesariamente tiene un contenido textual.

En este caso, veremos un ejemplo con _TF-IDF_. Comenzaremos definiendo el texto que vamos a resumir:

> Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico. Está considerado uno de los padres de la ciencia de la computación y precursor de la informática moderna. Proporcionó una influyente formalización de los conceptos de algoritmo y computación: la máquina de Turing. Formuló su propia versión que hoy es ampliamente aceptada como la tesis de Church-Turing (1936). Durante la segunda guerra mundial, trabajó en descifrar los códigos nazis, particularmente los de la máquina Enigma, y durante un tiempo fue el director de la sección Naval Enigma de Bletchley Park. Se ha estimado que su trabajo acortó la duración de esa guerra entre dos y cuatro años. Tras la guerra, diseñó uno de los primeros computadores electrónicos programables digitales en el Laboratorio Nacional de Física del Reino Unido y poco tiempo después construyó otra de las primeras máquinas en la Universidad de Mánchester. En el campo de la inteligencia artificial, es conocido sobre todo por la concepción de la prueba de Turing (1950), un criterio según el cual puede juzgarse la inteligencia de una máquina si sus respuestas en la prueba son indistinguibles de las de un ser humano.

In [ ]:
text = """Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico. Está considerado uno de los padres de la ciencia de la computación y precursor de la informática moderna. Proporcionó una influyente formalización de los conceptos de algoritmo y computación: la máquina de Turing. Formuló su propia versión que hoy es ampliamente aceptada como la tesis de Church-Turing (1936). Durante la segunda guerra mundial, trabajó en descifrar los códigos nazis, particularmente los de la máquina Enigma, y durante un tiempo fue el director de la sección Naval Enigma de Bletchley Park. Se ha estimado que su trabajo acortó la duración de esa guerra entre dos y cuatro años. Tras la guerra, diseñó uno de los primeros computadores electrónicos programables digitales en el Laboratorio Nacional de Física del Reino Unido y poco tiempo después construyó otra de las primeras máquinas en la Universidad de Mánchester. En el campo de la inteligencia artificial, es conocido sobre todo por la concepción de la prueba de Turing (1950), un criterio según el cual puede juzgarse la inteligencia de una máquina si sus respuestas en la prueba son indistinguibles de las de un ser humano."""

Vamos a dividir el texto por oraciones usando `spacy`:

In [ ]:
doc = nlp(text)
sents = list(map(lambda sent: sent.text, doc.sents))
display(sents)

Ahora, preprocesamos el texto:

In [ ]:
prep_sents = list(map(preprocess, sents))
display(prep_sents)

Extraemos una representación _TF-IDF_ de las oraciones:

In [ ]:
X = (
        TfidfVectorizer(norm=None)
        .fit_transform(prep_sents)
        .toarray()
        )
display(X)

Podemos saber qué tan importante es cada una de las oraciones al extraer la norma Euclidiana de cada uno de los vectores de documento:

In [ ]:
scores = np.linalg.norm(X, axis=1)
display(scores.size)

Como se puede ver, tenemos una importancia para cada una de las oraciones del texto. Vamos a crear un `DataFrame` con las oraciones y los scores para reordenar y extraer las oraciones más importantes del texto:

In [ ]:
scored_text = (
        pd.DataFrame({"text": sents, "score": scores})
        .sort_values(by="score", ascending=False)
        )
display(scored_text)

Un resumen extractivo podría estar dado por la oración más relevante:

In [ ]:
display(scored_text.text.head(1).values[0])

## **2. Léxicos**
---

Los léxicos son una forma de incorporar conceptos y semántica en las representaciones de bolsas de palabras. Este proceso consiste en estructurar léxicos (vocabularios) según determinados conceptos (categorías).

Un ejemplo típico de esto es [EmoLex](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm), se trata de un diccionario de palabras en inglés etiquetado en 10 distintas emociones.

Veamos cómo podemos cargar el _EmoLex_. Primero lo descargamos:

In [ ]:
!wget 'https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/emolex.json' -O 'emolex.json'

Ahora, lo cargamos con la librería `json`:

In [ ]:
import json
with open("emolex.json") as f:
    vocab = json.load(f)

Como se puede ver, tenemos distintos vocabularios para 10 conceptos distintos (emociones). Veamos de qué emociones disponemos:

In [ ]:
display(vocab.keys())

Podemos ver la cantidad de palabras por emoción:

In [ ]:
# Obtenemos las emociones:
emotions = list(vocab.keys())
# Obtenemos el número de palabras:
counts = [len(vocab[emotion]) for emotion in emotions]
# Diagrama de barras:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.bar(emotions, counts)
ax.set_xlabel("Emoción")
ax.set_ylabel("Número de palabras")
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
fig.show()

De igual forma, podemos ver algunas palabras asociadas a una emoción en específico:

In [ ]:
emotion = "joy"
display(vocab[emotion][:100])

Usando el léxico podemos conocer las emociones predominantes de un texto al mirar las coincidencias entre palabras. Veamos un ejemplo con el siguiente texto:
> La depresión es un trastorno del estado de ánimo que causa una persistente sensación de tristeza y pérdida de interés en actividades que solían ser disfrutables. La depresión puede afectar a la forma en que una persona se siente, piensa y se comporta, y puede interferir en su capacidad para llevar una vida normal. Los síntomas de la depresión pueden incluir cambios en el apetito, el sueño, la energía y la capacidad de concentración, así como sentimientos de desesperanza, irritabilidad y culpa. Si estás sufriendo de depresión, es importante que busques ayuda profesional.

In [ ]:
text = """La depresión es un trastorno del estado de ánimo que causa una persistente sensación de tristeza y pérdida de interés en actividades que solían ser disfrutables. La depresión puede afectar a la forma en que una persona se siente, piensa y se comporta, y puede interferir en su capacidad para llevar una vida normal. Los síntomas de la depresión pueden incluir cambios en el apetito, el sueño, la energía y la capacidad de concentración, así como sentimientos de desesperanza, irritabilidad y culpa. Si estás sufriendo de depresión, es importante que busques ayuda profesional."""

Creamos una función para contar coincidencias por emoción:

In [ ]:
def emotion_count(text, vocab):
    # Separamos las palabras por espacios.
    words = text.split(" ")
    # Creamos un diccionario donde se guardarán los conteos por cada emoción.
    counts = {
            emotion: sum(word in vocab[emotion] for word in words)
            for emotion in vocab
            }
    return counts

Aplicamos la función sobre el texto preprocesado para obtener la distribución de conceptos:

In [ ]:
counts = emotion_count(preprocess(text), vocab)
display(counts)

Podemos generar una visualización para mostrar la distribución de emociones (de acuerdo al léxico) del texto:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.bar(counts.keys(), counts.values())
ax.set_xlabel("Emoción")
ax.set_ylabel("Número de palabras")
ax.set_title("Distribución de emociones en el texto de ejemplo")
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
fig.show()

Como podemos ver, obtenemos una representación numérica basada en histogramas que es similar a las representaciones BoW. No obstante, en este caso estructuramos la información de acuerdo a conceptos predefinidos. Esto puede tener diversas ventajas al momento de solucionar tareas específicas, como por ejemplo, en manejo de textos clínicos con léxicos médicos, análisis de sentimientos con léxicos de emociones, entre otros.

Veamos otro ejemplo con el siguiente texto:

> La felicidad es un estado emocional que se caracteriza por sentir satisfacción y bienestar. La felicidad puede ser el resultado de tener relaciones saludables y positivas, lograr metas importantes, experimentar cosas nuevas y emocionantes, o simplemente disfrutar de las cosas simples de la vida. La felicidad es subjetiva y puede significar cosas diferentes para diferentes personas. Algunas personas pueden encontrar la felicidad en el éxito profesional, mientras que para otros puede ser más importante tener una buena salud o disfrutar de actividades al aire libre. Lo importante es encontrar lo que te hace feliz y hacerlo una parte integral de tu vida.

In [ ]:
text = """La felicidad es un estado emocional que se caracteriza por sentir satisfacción y bienestar. La felicidad puede ser el resultado de tener relaciones saludables y positivas, lograr metas importantes, experimentar cosas nuevas y emocionantes, o simplemente disfrutar de las cosas simples de la vida. La felicidad es subjetiva y puede significar cosas diferentes para diferentes personas. Algunas personas pueden encontrar la felicidad en el éxito profesional, mientras que para otros puede ser más importante tener una buena salud o disfrutar de actividades al aire libre. Lo importante es encontrar lo que te hace feliz y hacerlo una parte integral de tu vida."""

Veamos los conteos de emociones:

In [ ]:
counts = emotion_count(preprocess(text), vocab)
display(counts)

Podemos generar una visualización para mostrar la distribución de emociones (de acuerdo al léxico) del texto:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.bar(counts.keys(), counts.values())
ax.set_xlabel("Emoción")
ax.set_ylabel("Número de palabras")
ax.set_title("Distribución de emociones en el texto de ejemplo")
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
fig.show()

## **3. Nubes de Palabras**
---

Una de las herramientas más útiles para el entendimiento de información textual son las nubes de palabras. Se trata de un tipo de visualización donde mostramos la relevancia (puede calcularse como los conteos, _TF-IDF_ u otras) de una palabra tal y como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=16oFlir07J0ULy9jr7nbPnkTtFwd6C_1_" width="80%">



### **3.1. Corpus y Preprocesamiento**
---

En este ejemplo usaremos el dataset [Language Detection de Kaggle](https://www.kaggle.com/datasets/basilb2s/language-detection) que usamos en el notebook de introducción a las bolsas de palabras. Comencemos descargándolo y manipulándolo con `pandas` para extraer únicamente los textos en español:

In [ ]:
data = (
        pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/language.csv")
        .query("language == 'Spanish'")
        )

display(data.head())

Preprocesamos el corpus:

In [ ]:
corpus_prep = data.text.apply(preprocess).to_list()
display(corpus_prep[:5])

### **3.2. WordCloud**
---

Desde _Python_ podemos construir nubes de palabras con la librería `wordcloud`. Veamos cómo instarla:

In [ ]:
!pip install wordcloud

Para generar la visualización, podemos importar la clase `WordCloud`:

In [ ]:
from wordcloud import WordCloud

Vamos a generar una nube de palabras a partir del `CountVectorizer` de `sklearn`, para ello usaremos el corpus en español que habíamos filtrado anteriormente.

Entrenamos un `CountVectorizer`:

In [ ]:
vect = (
    CountVectorizer(max_features=1000, max_df=0.7)
    .fit(corpus_prep)
    )

Extraemos la representación de bolsa de palabras y el vocabulario:

In [ ]:
X = vect.transform(corpus_prep)
vocab = vect.get_feature_names_out()

Vamos a generar un conteo completo de cada palabra en el corpus (no por documento) al sumar sobre la matriz de bolsa de palabras:

In [ ]:
counts = np.array(X.sum(axis=0)).flatten()
display(counts)

Para generar la nube de palabras, debemos crear un diccionario donde las claves sean las palabras y los valores las importancias:

In [ ]:
counts_dict = {word: count for word, count in zip(vocab, counts)}
display(counts_dict)

Generamos la nube de palabras:

In [ ]:
wc = (
        WordCloud( width=500, height=300)
        .generate_from_frequencies(counts_dict)
        )
display(wc)

Finalmente, la visualizamos:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(wc)
ax.axis("off")
fig.show()

Podemos cambiar algunos aspectos en la configuración de la nube de palabras, por ejemplo:

- `font_path`: permite cambiar el tipo de letra.
- `width`: ancho de la imagen.
- `height`: alto de la imagen.
- `prefer_horizontal`: una proporción que define la preferencia de las palabras para aparecer de forma vertical (>1) u horizontal (<1).
- `mask`: máscara binaria indicando dónde se deben mostrar las palabras.
- `max_words`: número máximo de palabras a mostrar.
- `mode`: define el tipo de espacio de color a usar, por ejemplo `"RGBA"` permite transparencia en el fondo.
- `background_color`: color para usar en el fondo de la imagen.
- `colormap`: paleta de colores dentro de las [disponibles](https://matplotlib.org/stable/gallery/color/colormap_reference.html) de `matplotlib`.

Veamos un ejemplo un poco más elaborado. Primero descargamos una imagen para usar como máscara:

In [ ]:
!wget 'https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/colombia.jpg' -O 'colombia.jpg'

Ahora, cargamos la imagen usando la librería `cv2` (librería para procesamiento de imágenes):

In [ ]:
import cv2
im = cv2.resize(cv2.imread("colombia.jpg", 0), (1000, 1000))
display(im)

Podemos visualizarla:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(im, cmap="gray")
ax.axis("off")
fig.show()

Ahora, generamos la nube de palabras:

In [ ]:
wc = WordCloud(
        mask = im,
        colormap = "Blues",
        background_color = "#FFFFFF" # color blanco en hex
        ).generate_from_frequencies(counts_dict)

Veamos la nube:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(wc)
ax.axis("off")
fig.savefig("wordcloud.png")

Como pudimos ver, las nubes de palabras son un tipo de visualización bastante entendible que permiten mostrar de mejor forma las representaciones basadas en bolsas de palabras.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Understanding Automatic Text Summarization](https://towardsdatascience.com/understanding-automatic-text-summarization-1-extractive-methods-8eb512b21ecc).
- [Generating word cloud in Python](https://www.datacamp.com/tutorial/wordcloud-python).

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*